## ĐỒ ÁN CUỐI KÌ
****

### Thành viên
- 19127642&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Võ Nhất Huy
- 19127592&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Lê Minh Trí
### Dataset
- HR Analytics: Job Change of Data Scientists
- Link: https://www.kaggle.com/arashnic/hr-analytics-job-change-of-data-scientists
***

### Các thư viện cần thiết

In [127]:
import pandas as pd
import numpy as np

### Đọc file và khám phá dữ liệu

In [128]:
aug_df = pd.read_csv('aug_train.csv')

In [129]:
aug_df

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,city_173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,14,NaN,NaN,1,42,1.0
19154,31398,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,NaN,NaN,4,52,1.0
19155,24576,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,44,0.0
19156,5756,city_65,0.802,Male,Has relevent experience,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,97,0.0


#### Đếm số dòng/cột
- Số dòng và số cột được lưu vào hai biến `n_rows` và `n_columns`

In [130]:
n_rows = len(aug_df)
n_columns = len(aug_df.columns)
print(n_rows, n_columns)

19158 14


In [131]:
aug_df.dtypes

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
target                    float64
dtype: object

#### Ý nghĩa của mỗi cột

In [136]:
with open('description.txt', 'r',encoding='utf-8') as f:
    print(f.read())

1) enrollee_id:                 Unique ID for candidate
2) city:                        City code
3) city_ development _index:    Developement index of the city (scaled)
4) gender:                      Gender of candidate
5) relevent_experience:         Relevant experience of candidate
6) enrolled_university:         Type of University course enrolled if any
7) education_level:             Education level of candidate
8) major_discipline:            Education major discipline of candidate
9) experience:                  Candidate total experience in years
10) company_size:               No of employees in current employer's company
11) company_type:               Type of current employer
12) lastnewjob:                 Difference in years between previous job and current job
13) training_hours:             training hours completed
14) target:                     0 – Not looking for job change, 1 – Looking for a job change


Giáo dục:
- enrolled_university
- education_level
- major_discipline

Môi trường làm việc:
- city/city_ development _index*
- relevent_experience
- company_size
- company_type

Yếu tố cá nhân khác:
- Gender
- experience
- lastnewjob
- training_hours

#### Kiểm tra dòng bị lặp

In [137]:
print(aug_df.duplicated().any())

False


#### Kiểm tra số giá trị bị thiếu trong mỗi cột
- Ta lưu giá trị số dòng bị thiếu của mỗi cột vào dictionary `missing_dict`. 
- Trong đó, key là tên của cột, value là số dòng bị thiếu trong cột đó.
- Ta có thể in thêm % số dòng bị thiếu, để ước tính mức độ thiếu dữ liệu, từ đó đưa ra quyết định có nên drop cột đó hay không

In [133]:
column_name = aug_df.columns
col_missing_count = []
for col in column_name:
    col_missing_count.append(aug_df[col].isna().sum())
missing_dict = dict(zip(column_name, col_missing_count))
for key, value in missing_dict.items():
    print(key, ':', value, '|', value/n_rows*100)

enrollee_id : 0 | 0.0
city : 0 | 0.0
city_development_index : 0 | 0.0
gender : 4508 | 23.530639941538784
relevent_experience : 0 | 0.0
enrolled_university : 386 | 2.0148240943731075
education_level : 460 | 2.401085708320284
major_discipline : 2813 | 14.68316108153252
experience : 65 | 0.33928385008873574
company_size : 5938 | 30.994884643490973
company_type : 6140 | 32.04927445453596
last_new_job : 423 | 2.207954901346696
training_hours : 0 | 0.0
target : 0 | 0.0


### Tiền xử lí dữ liệu
- Đối với các cột dữ liệu bị thiếu, sau khi quan sát các giá trị ở trong bảng dữ liệu, chúng em nhận ra ở đây tồn tại hai vấn đề:
    - Dữ liệu trong các cột bị thiếu không hề đồng nhất. Đúng hơn là ở các cột mà nhìn vào ta nghĩ nó là cột `numeric` thì không phải, do nó có chứa các giá trị như `<1, >4, 1000+, ...`, hay nói đúng hơn là chứa các miền giá trị
    - Các miền giá trị có cách ghi không đồng nhất (company_size là trường hợp này)
- Do đó, chúng em quyết định tách các cột này ra thành 2 loại để xử lí: cột không chứa miền giá trị (chỉ chứa các string) và cột chứa số có miền giá trị

#### Các cột chỉ chứa string
- Đối với các cột loại này, cách xử lí tương đối là đơn giản, chỉ cần thay các giá trị bị thiếu bằng mode, hoặc một giá trị khác là được

In [134]:
def fill_missing_by_mode(column_name):
    if column_name == 'Gender':
        aug_df[column_name].fillna('Other')
    else:
        mode = aug_df[column_name].mode(dropna=True)
        aug_df[column_name].fillna(mode)



#### Các cột chứa miền giá trị 
- Xử lí các cột này có một chút rắc rối, nhưng nhìn chung vẫn tương đối dễ thực hiện.
- Như đã đề cập ở trên, do cột company_size khá là đặc biệt, cho nên ta lại tiếp tục tách riêng ra để xử lí.
- Đối với các cột khác, ta chỉ cần thay thể các giá trị chứa dấu `<` hoặc `>` bằng số được giảm/tăng 1 đơn vị tương ứng
- Còn đồi với cột company_size, ta có vài thứ cần thực hiện như sau:
    - Giá trị `10/49` trong cột có nghĩa là `10-49` giống như các giá trị khác, chứ không phải là tháng 10 năm 49 :)))
    - Thay thế các giá trị ở dạng `a-b` bằng giá trị trung vị của nó (đưa về int tất nhiên sẽ dễ xử lí hơn)
    - Đôi với giá trị `<10`, ta không thể để = 0 được (mặc dù về mặt hình thức thì ta để bất kì số nào trong khoảng 0-9 đều như nhau), vì như thế ý nghĩa của giá trị này sẽ mất đi.
    - Đối với giá trị `10000+`, ta chỉ cần xóa dấu `+` đi là được
- Sau khi xác định xong các giá trị cần thay thế, ta tiến hành thay trực tiếp giá trị đó vào cột (dùng `loc`)

In [135]:
def fill_missing_undefinite_range(column_name): 
    list_unique = aug_df[column_name].dropna().unique()
    list_number = [s for s in list_unique if s.isdigit()]
    list_number = list(map(int, list_number)); list_number.sort()
    list_range = [s for s in list_unique if not(s.isdigit())]
    mid = len(list_number) // 2
    median = (list_number[mid] + list_number[~mid]) / 2
    print(list_unique,list_number, list_range)
    aug_df[column_name]=aug_df[column_name].fillna(median)
    if 'never' in list_range:
        aug_df.loc[aug_df[column_name].str.contains('never', na=False), column_name] = 0
    aug_df.loc[aug_df[column_name].str.contains('>', na=False), column_name] = list_number[-1]
    aug_df.loc[aug_df[column_name].str.contains('<', na=False), column_name] = list_number[0]

def fill_missing_definite_range(): #company_size
    list_unique = aug_df['company_size'].dropna().unique()
    list_replace = len(list_unique)*[None]
    for i in range(len(list_unique)):
        if list_unique[i] == '10/49':
            list_replace[i] = (10+50)/2
        elif '+' in list_unique[i]:
            list_replace[i] = int(list_unique[i][:-1])
        elif list_unique[i] == '<10':
            list_replace[i] = 1
        else:
            x = list_unique[i].split("-", 1)
            list_replace[i] = (int(x[0])+int(x[1])+1)*0.2
    mid = len(list_replace) // 2
    median = (list_replace[mid] + list_replace[~mid]) / 2      
    aug_df[column_name]=aug_df[column_name].fillna(median)
    for i in range(len(list_replace)):
        aug_df.loc[aug_df['company_size'].str.contains(list_unique[i], na=False), 'company_size'] = list_replace[i]

print(aug_df['company_size'])
print(aug_df['company_size'].isna().sum())
fill_missing_definite_range()
print(aug_df['company_size'])
print(aug_df['company_size'].isna().sum())

0            NaN
1          50-99
2            NaN
3            NaN
4          50-99
          ...   
19153        NaN
19154        NaN
19155      50-99
19156    500-999
19157        NaN
Name: company_size, Length: 19158, dtype: object
5938
0        2100.0
1          30.0
2        2100.0
3        2100.0
4          30.0
          ...  
19153    2100.0
19154    2100.0
19155      30.0
19156     300.0
19157    2100.0
Name: company_size, Length: 19158, dtype: object
0


### Đặt câu hỏi
1. Xác định các nhân tố khách quan, chủ quan ảnh hưởng tới kết quả của cột target. Lí giải vì sao nhân tố đó lại là khách quan, chủ quan.
2. Dựa vào các nhân tố khách quan, đánh giá xem chúng có ảnh hưởng như thế nào tới cột target? 
3. Dựa vào các nhân tố chủ quan, đánh giá xem chúng có ảnh hưởng như thế nào tới cột target?

### Trả lời câu hỏi